In [1]:
import yfinance as yf
import pandas as pd
import numpy as np

In [3]:
df = yf.download('AAPL',start='1999-12-31', end='2010-12-31',progress=True)
df = df.loc[:,['Adj Close']]
df.rename(columns = {'Adj Close': 'adj_close'},inplace=True)

[*********************100%***********************]  1 of 1 completed


## Calculate simple returns:
### Simple Return = (CurrentValue-PrevValue)/PrevValue
#### Rt = (Pt-Pt-1)/Pt-1

In [4]:
df['simple_rtn'] = df.adj_close.pct_change()
# Calculate log returns:  ln(CurrentValue/PrevValue)
df['log_rtn'] = np.log(df.adj_close/df.adj_close.shift(1))
df

,adj_close,simple_rtn,log_rtn
Date,,,
1999-12-31,3.173011,NaN,NaN
2000-01-03,3.454628,0.088754,0.085034
2000-01-04,3.163368,-0.084310,-0.088078
2000-01-05,3.209661,0.014634,0.014528
2000-01-06,2.931901,-0.086539,-0.090514
...,...,...,...
2010-12-23,39.947918,-0.004798,-0.004809
2010-12-27,40.081257,0.003338,0.003332
2010-12-28,40.178780,0.002433,0.002430


In [5]:
import quandl
API_KEY = '8ohVvwCgmzgRpFeza8FD'
quandl.ApiConfig.api_key = API_KEY

In [19]:
df_all_dates = pd.DataFrame(index=pd.date_range(start='1999-12-31', end='2010-12-31'))

In [20]:
df= df_all_dates.join(df['adj_close'], how='left') \
    .fillna(method='ffill') \
    .asfreq('M')

In [22]:
df

,adj_close
1999-12-31,3.173011
2000-01-31,3.201945
2000-02-29,3.537570
2000-03-31,4.191461
2000-04-30,3.828833
...,...
2010-08-31,30.010326
2010-09-30,35.028503
2010-10-31,37.155525
2010-11-30,38.411003


In [23]:
df_cpi = quandl.get(dataset='RATEINF/CPI_USA',
                   start_date='1999-12-01',
                   end_date='2010-12-31')
df_cpi.rename(columns={'Value' : 'cpi'},inplace=True)
df_cpi

,cpi
Date,
1999-12-31,168.300
2000-01-31,168.800
2000-02-29,169.800
2000-03-31,171.200
2000-04-30,171.300
...,...
2010-08-31,218.312
2010-09-30,218.439
2010-10-31,218.711


In [26]:
df_merged = df.join(df_cpi,how='left')
# df_merged['simple_rtn'] = df_merged.adj_close.pct_change()
df_merged

,adj_close,cpi
1999-12-31,3.173011,168.300
2000-01-31,3.201945,168.800
2000-02-29,3.537570,169.800
2000-03-31,4.191461,171.200
2000-04-30,3.828833,171.300
...,...,...
2010-08-31,30.010326,218.312
2010-09-30,35.028503,218.439
2010-10-31,37.155525,218.711
2010-11-30,38.411003,218.803


In [30]:
df_merged['simple_rtn'] = df_merged.adj_close.pct_change()
df_merged['inflation_rate'] = df_merged.cpi.pct_change()

df_merged['real_rtn'] = (df_merged.simple_rtn + 1) / (df_merged.inflation_rate + 1) - 1

In [31]:
df_merged

,adj_close,cpi,simple_rtn,inflation_rate,real_rtn
1999-12-31,3.173011,168.300,NaN,NaN,NaN
2000-01-31,3.201945,168.800,0.009119,0.002971,0.006130
2000-02-29,3.537570,169.800,0.104819,0.005924,0.098312
2000-03-31,4.191461,171.200,0.184842,0.008245,0.175153
2000-04-30,3.828833,171.300,-0.086516,0.000584,-0.087049
...,...,...,...,...,...
2010-08-31,30.010326,218.312,-0.055005,0.001381,-0.056308
2010-09-30,35.028503,218.439,0.167215,0.000582,0.166536
2010-10-31,37.155525,218.711,0.060723,0.001245,0.059403
2010-11-30,38.411003,218.803,0.033790,0.000421,0.033355
